In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pywt
import pyeeg
import nolds
import math
import numpy as np
import warnings
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics

In [2]:
#timefrequency domain features
#get wavelet transform
def DWT (x):
    wt_trans=pywt.dwt(x,'db4')
    return wt_trans
#time domain features
#Hjorth features
def Hjorth(x):
    hj=pyeeg.hjorth(x)
    return hj
#detrended fluctuation analysis
def DFA(x):
    dfa=pyeeg.dfa(x)
    return np.array(dfa)
#first order difference
def FirstDiff(x):
    fd=pyeeg.first_order_diff(x)
    return fd
#Hurst exponent feature
def Hurst(x):
    hur=pyeeg.hurst(x)
    return np.array(hur)
#statistical features
def stats(x):
    mean=np.mean(x)
    sd=np.std(x)
    return np.array((mean,sd))
#petrosian fractal dimension 
def PFD(x):
    pfd=pyeeg.pfd(x)
    return np.array(pfd)
#Power features (fft - fourier transform)
def Power(x):
    F=np.fft.fft(x)
    P=F*np.conjugate(F)
    power=sum(P)
    pr=math.sqrt(power.real*power.real + power.imag*power.imag)
    return np.array(pr) 
#correlation dimension features
def CorrelationDimension(x):
    cd=nolds.corr_dim(x,1)
    return np.array(cd)
#frequency domain features
#band power for each frequency
def Bandpower(x):
    fs=128
    band=[1,4,8,12,30]
    bp=pyeeg.bin_power(x,band,fs)
    return bp
def SpectralEntropy(x):
    fs=128
    band=[1,4,8,12,30]
    b=pyeeg.bin_power(x,band,fs)
    se=pyeeg.spectral_entropy(x,band,fs,Power_Ratio=b)
    se=[0 if math.isnan(x) else x for x in se]
    return se
import tsfel
def get_tsfel_features(ts,fs):
    cfg = tsfel.get_features_by_domain()
    return tsfel.time_series_features_extractor(cfg, ts, fs, verbose=False).values[0]
def get_features(train):
    features=[]
    for i in np.arange(0,len(train),1):
        print(i)
        warnings.filterwarnings("ignore")
        data=train.iloc[i,:]
        wt=DWT(data)[0]
        hj=Hjorth(data)
        dfa=DFA(data).reshape(1,)
        hurst=Hurst(data).reshape(1,)
        mean=stats(data)[0].reshape(1,)
        sd=stats(data)[1].reshape(1,)
        pfd=PFD(data).reshape(1,)
        power=Power(data).reshape(1,)
        cd=CorrelationDimension(data).reshape(1,)
        bp=Bandpower(data)
        se=SpectralEntropy(data)
        tsfel_fts =get_tsfel_features(data,128)
        ft = np.concatenate([wt,hj,dfa,hurst,mean,sd,pfd,power,cd,bp[0],bp[1],se,tsfel_fts])
        features.append(ft)
    features = pd.DataFrame(features)
    return(features)

In [3]:
def get_wt(train):
    features=[]
    for i in np.arange(0,len(train),1):
        warnings.filterwarnings("ignore")
        data=train.iloc[i,:]
        wt=DWT(data)[1]
        ft = np.concatenate([wt])
        features.append(ft)
    features = pd.DataFrame(features)
    return(features)

In [4]:
def get_last_stage(train):
    features=[]
    for i in np.arange(0,len(train),1):
        warnings.filterwarnings("ignore")
        if i<2:
            ls_1=train.iloc[i,:]
            ls_2=train.iloc[i,:]
            ls_3=train.iloc[(i+1),:]
            ls_4=train.iloc[(i+2),:]
        elif i<(len(train)-2):
            ls_1=train.iloc[(i-1),:]
            ls_2=train.iloc[(i-2),:]
            ls_3=train.iloc[(i+1),:]
            ls_4=train.iloc[(i+2),:]
        else:
            ls_1=train.iloc[(i-1),:]
            ls_2=train.iloc[(i-2),:]
            ls_3=train.iloc[(i),:]
            ls_4=train.iloc[(i),:]
        ft = np.concatenate([ls_1,ls_2,ls_3,ls_4])
        features.append(ft)
    features = pd.DataFrame(features)
    return(features)

In [5]:
def bandpower(data, sf, band, method='welch', window_sec=None, relative=False):
    from scipy.signal import welch
    from scipy.integrate import simps
    from mne.time_frequency import psd_array_multitaper

    band = np.asarray(band)
    low, high = band

    # Compute the modified periodogram (Welch)
    if method == 'welch':
        if window_sec is not None:
            nperseg = window_sec * sf
        else:
            nperseg = (2 / low) * sf

        freqs, psd = welch(data, sf, nperseg=nperseg)

    elif method == 'multitaper':
        psd, freqs = psd_array_multitaper(data, sf, adaptive=True,
                                          normalization='full', verbose=0)

    # Frequency resolution
    freq_res = freqs[1] - freqs[0]

    # Find index of band in frequency vector
    idx_band = np.logical_and(freqs >= low, freqs <= high)

    # Integral approximation of the spectrum using parabola (Simpson's rule)
    bp = simps(psd[idx_band], dx=freq_res)

    if relative:
        bp /= simps(psd, dx=freq_res)
    return bp
def get_emg_fts(emg):
    """
    MAV (mean absolute value);
    ZC (zero crossing);
    WL (waveform length)
    """
    mav = sum(abs(emg))/len(emg)
    zc = 0
    for i in np.arange(0,len(emg)-1,1):
        if emg[i]*emg[i+1] < 0:
            zc = zc + 1
    wl = 0
    for i in np.arange(0,len(emg)-1,1):
        wl = wl + abs(emg[i+1]-emg[i])
    return([mav, zc, wl])
def eeg_band_power(eeg, freq):
    delta = bandpower(eeg, freq, [0.5, 4], 'multitaper')
    theta = bandpower(eeg, freq, [4, 8], 'multitaper')
    alpha = bandpower(eeg, freq, [8, 12], 'multitaper')
    beta = bandpower(eeg, freq, [12, 30], 'multitaper')
    # delta_beta ratio
    db_ratio = bandpower(eeg, freq, [0.5, 4], 'multitaper') / bandpower(eeg, freq, [12, 30], 'multitaper')
    return([delta,theta,alpha,beta,db_ratio])

In [6]:
def extract_ft(eeg1, eeg2, emg, freq):
    emg_fts = get_emg_fts(emg)
    eeg1_bp = eeg_band_power(eeg1, freq)
    eeg2_bp = eeg_band_power(eeg2, freq)
    tmp = np.concatenate([emg_fts,eeg1_bp, eeg2_bp])
    return tmp

In [ ]:
ft_mat = []
for i in np.arange(0,len(train_eeg1),1):
    tmp = extract_ft(train_eeg1.iloc[i,:],train_eeg2.iloc[i,:],train_emg.iloc[i,:],128)
    ft_mat.append(tmp)
ft_mat = pd.DataFrame(ft_mat)
ft_mat.to_csv('ft_mat.csv', index=True)

In [7]:
train_eeg1=pd.read_csv("task data/train_eeg1.csv")
train_eeg2=pd.read_csv("task data/train_eeg2.csv")
train_emg=pd.read_csv("task data/train_emg.csv")
train_eeg1=train_eeg1.set_index('Id')
train_eeg2=train_eeg2.set_index('Id')
train_emg=train_emg.set_index('Id')

In [8]:
train1=pd.read_csv("train1.csv",index_col=0)
train2=pd.read_csv("train2.csv",index_col=0)
train3=pd.read_csv("train3.csv",index_col=0)
y_train=pd.read_csv("train_labels.csv")
y_train=y_train.set_index('Id')

In [9]:
train1=train1.iloc[:,260:670]
train2=train2.iloc[:,260:670]
train3=train3.iloc[:,260:670]

In [10]:
ft_mat=pd.read_csv("ft_mat.csv",index_col=0)

In [11]:
pd.set_option('use_inf_as_na', True)
train1.replace([np.inf,-np.inf], np.nan, inplace=True)
train1 = train1.loc[:,train1.median().notna()]
train1= train1.fillna(train1.median())
train2.replace([np.inf,-np.inf], np.nan, inplace=True)
train2 = train2.loc[:,train2.median().notna()]
train2= train2.fillna(train2.median())
train3.replace([np.inf,-np.inf], np.nan, inplace=True)
train3 = train3.loc[:,train3.median().notna()]
train3= train3.fillna(train3.median())

In [12]:
frames = [train1,train2,train3,ft_mat]
x_train = pd.concat(frames,axis=1)
print(x_train.shape)

(64800, 1243)


In [13]:
ls_all=['X%d' % i for i in range(0, x_train.shape[1], 1)]
x_train.columns=ls_all
#x_train.to_csv('x_train_ext.csv', index=True)

In [14]:
sd_values = x_train.std()
cols_to_delete=np.where(sd_values==0)[0]
x_train=x_train.drop(x_train.columns[cols_to_delete],axis=1)
x_train.shape

(64800, 1189)

In [15]:
sd_values = x_train.std()
mean_values = x_train.mean()
x_train_normalised = (x_train - mean_values)/sd_values

In [30]:
from sklearn.feature_selection import SelectKBest,f_classif
selector = SelectKBest(f_classif, k=50)
X_new = selector.fit_transform(x_train_normalised, y_train)
mask_k = selector.get_support()
selected_cols = x_train_normalised.columns[mask_k]
X_new = pd.DataFrame(X_new)
X_new.columns = selected_cols

In [31]:
ft1_laststage=get_last_stage(X_new.iloc[0:21600,:])
ft2_laststage=get_last_stage(X_new.iloc[21600:43200,:])
ft3_laststage=get_last_stage(X_new.iloc[43200:64800,:])
frames = [ft1_laststage,ft2_laststage,ft3_laststage]
ft_laststage = pd.concat(frames,ignore_index=True)

In [32]:
frames = [X_new,ft_laststage]
x_final = pd.concat(frames,axis=1)

In [33]:
x_final.shape

(64800, 250)

In [34]:
X_train=x_final.iloc[21600:64800,:]
X_val=x_final.iloc[0:21600,:]
Y_train=y_train.iloc[21600:64800,:]
Y_val=y_train.iloc[0:21600,:]

In [35]:
model = SVC(class_weight='balanced',gamma=0.02,C=0.1)
model.fit(X_train,Y_train)
preds=model.predict(X_val)
metrics.balanced_accuracy_score(Y_val,preds)

0.9084749861035429

In [36]:
X_train,X_val,Y_train,Y_val=train_test_split(x_final,y_train,test_size=0.2,stratify=y_train,shuffle=True)

In [37]:
model1 = SVC(class_weight='balanced',gamma=0.02,C=0.1)
model1.fit(X_train,Y_train)
preds=model1.predict(X_val)
metrics.balanced_accuracy_score(Y_val,preds)

0.9618500401125706

In [27]:
test_eeg1=pd.read_csv("task data/test_eeg1.csv")
test_eeg2=pd.read_csv("task data/test_eeg2.csv")
test_emg=pd.read_csv("task data/test_emg.csv")
test_eeg1=test_eeg1.set_index('Id')
test_eeg2=test_eeg2.set_index('Id')
test_emg=test_emg.set_index('Id')

In [28]:
test1=pd.read_csv("test1.csv",index_col=0)
test2=pd.read_csv("test2.csv",index_col=0)
test3=pd.read_csv("test3.csv",index_col=0)

In [29]:
test1=test1.iloc[:,260:670]
test2=test2.iloc[:,260:670]
test3=test3.iloc[:,260:670]

In [30]:
ft_test_mat=pd.read_csv("ft_test_mat.csv",index_col=0)

In [31]:
pd.set_option('use_inf_as_na', True)
test1.replace([np.inf,-np.inf], np.nan, inplace=True)
test1 = test1.loc[:,test1.median().notna()]
test1= test1.fillna(test1.median())
test2.replace([np.inf,-np.inf], np.nan, inplace=True)
test2 = test2.loc[:,test2.median().notna()]
test2= test2.fillna(test2.median())
test3.replace([np.inf,-np.inf], np.nan, inplace=True)
test3 = test3.loc[:,test3.median().notna()]
test3= test3.fillna(test3.median())
frames_t = [test1,test2,test3,ft_test_mat]
x_test = pd.concat(frames_t,axis=1)

In [32]:
x_test.columns=ls_all
#x_test.to_csv('x_test.csv', index=True)
x_test=x_test.drop(x_test.columns[cols_to_delete],axis=1)
print(x_test.shape)
x_test_normalised = (x_test - mean_values)/sd_values
X_test=x_test_normalised.loc[:,list(selected_cols)]

(43200, 1189)


In [33]:
ft1_laststage_t=get_last_stage(X_test.iloc[0:21600,:])
ft2_laststage_t=get_last_stage(X_test.iloc[21600:43200,:])
ft3_laststage_t=get_last_stage(X_test.iloc[43200:64800,:])
frames = [ft1_laststage_t,ft2_laststage_t,ft3_laststage_t]
ft_laststage_t = pd.concat(frames,ignore_index=True)

In [34]:
frames = [X_test,ft_laststage_t]
x_test_final = pd.concat(frames,axis=1)

In [35]:
column_names=['y']
y_test=pd.DataFrame(columns=column_names)
y_test['Id']=x_test_normalised.index.values

In [36]:
x_test_final.shape

(43200, 250)

In [38]:
y_test=y_test.set_index('Id')

In [53]:
pred=model1.predict(x_test_final)
y_test.iloc[:,0]=pred
y_test.to_csv('SVC_RS_1.csv', index=True,float_format='%.3f')  

In [ ]:
y1=pd.read_csv("SVC_LOM_1.csv",index_col=0)
y2=pd.read_csv("SVC_LOM_2.csv",index_col=0)
y3=pd.read_csv("SVC_LOM_3.csv",index_col=0)
y4=pd.read_csv("SVC_RS_1.csv",index_col=0)
y_combined=pd.concat([y1,y2,y3,y4],axis=1)

In [73]:
y_softmax=[]
for i in np.arange(0,len(y_combined),1):
    data=np.array(y_combined.iloc[i,:])
    counts = np.bincount(data)
    soft_max=int(np.argmax(counts))
    y_softmax=np.append(y_softmax,soft_max)

In [74]:
y_test.iloc[:,0]=y_softmax
y_test.to_csv('SVC_softmax.csv', index=True,float_format='%.3f')  
y_test

,y
Id,
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
...,...
43195,1.0
43196,1.0
43197,1.0
